In [3]:
from PIL import Image
import glob,os
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np 
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches

import os                                                                                                                                                                               
import logging
from Pegasus.api import *


from utils.wf import split_data_filenames, create_ann_list, create_preprocessed_filelist

# use this to get path to your current dir and get absolute paths to your files
filename = "img.png"
print(os.path.join(os.getcwd(),filename))

/home/scitech/shared-data/face-recognition-wf/img.png


In [4]:
# When you restart the docker make sure to reinstall these:
#! sudo python3 -m pip install --upgrade pip
#! sudo python3 -m pip install --upgrade Pillow numpy torchvision matplotlib
#! sudo python3 -m pip install pandas torch bs4

In [5]:
imagesList = glob.glob('Images2/*.png')
annotationList = glob.glob('annotations2/*.xml')

In [6]:
train_filenames,val_filenames,test_filenames, files_split_dict = split_data_filenames(imagesList)

In [7]:
train_imgs, train_ann = create_ann_list(train_filenames)
val_imgs, val_ann     = create_ann_list(val_filenames)
test_imgs, test_ann   = create_ann_list(test_filenames)

train_preprocessed_files = create_preprocessed_filelist(train_filenames)
val_preprocessed_files   = create_preprocessed_filelist(val_filenames)
test_preprocessed_files  = create_preprocessed_filelist(test_filenames)

In [8]:
rc = ReplicaCatalog()

inputFiles = []
for img in imagesList:
    fileName = img.split("/")[1]
    img_file = File(fileName)
    inputFiles.append(img_file)
    rc.add_replica("local", img_file,  os.path.join(os.getcwd(),str(img)))
    
annFiles = []
for ann in annotationList:
    fileName = ann.split("/")[1]
    ann_file = File(fileName)
    annFiles.append(ann_file)
    rc.add_replica("local", ann_file,  os.path.join(os.getcwd(),str(ann)))
rc.write()

In [9]:
# Add Transformations

preprocess_imgs = Transformation(
                "preprocess_images",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/img_norm.py"),
                is_stageable=True
            )


'''model_hpo = Transformation(
                "hpo",
                site="local",
                pfn=  os.path.join(os.getcwd(),"bin/model_hpo.py"),
                is_stageable=True
            )'''

'model_hpo = Transformation(\n                "hpo",\n                site="local",\n                pfn=  os.path.join(os.getcwd(),"bin/model_hpo.py"),\n                is_stageable=True\n            )'

In [10]:
tc = TransformationCatalog()
tc.add_transformations(preprocess_imgs)#, model_hpo)
tc.write()

In [11]:
#Create workflow

wf = Workflow("mask_detection_workflow")


# Add jobs to the workflow

preprocess_train_job = Job(preprocess_imgs)
preprocess_train_job.add_inputs(*train_imgs)
preprocess_train_job.add_outputs(*train_preprocessed_files)


preprocess_val_job = Job(preprocess_imgs)
preprocess_val_job.add_inputs(*val_imgs)
preprocess_val_job.add_outputs(*val_preprocessed_files)


preprocess_test_job = Job(preprocess_imgs)
preprocess_test_job.add_inputs(*test_imgs)
preprocess_test_job.add_outputs(*test_preprocessed_files)


#hpo_job = Job(model_hpo)
#hpo_job.add_inputs(*train_preprocessed_files, *val_preprocessed_files)
#hpo_job.add_inputs(hpo_results)

In [12]:
# add jobs to workflow
wf.add_jobs(
    preprocess_train_job,
    preprocess_val_job,
    preprocess_test_job
)

# run workflow
try:
    wf.plan(submit=True)
    wf.wait()
    wf.statistics()
except PegasusClientError as e:
    print(e.output)

[##################################################] 100.0% ..Success (Completed: 11, Queued: 0, Running: 0, Failed: 0)
